In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics roboflow pyyaml


Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
import sys
import gc
import torch
import torch.nn as nn
import yaml

from ultralytics import YOLO
from ultralytics.nn import tasks, modules
from roboflow import Roboflow


In [3]:
# ======================================================================================
# 1. CBAM 모듈 정의
# ======================================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)


class CBAM_Universal(nn.Module):
    """
    YOLO 커스텀 모듈로 쓰기 위한 Universal CBAM.
    YAML에서 [ 'CBAM_Universal', [channels] ] 형태로 호출됨.
    """
    def __init__(self, *args, **kwargs):
        super(CBAM_Universal, self).__init__()
        c1 = args[0]  # in_channels
        k = 7
        print(f"✅ CBAM 생성됨 | 적용 채널: {c1}")
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(k)

    def forward(self, x):
        out = self.channel_attention(x) * x
        out = self.spatial_attention(out) * out
        return out


# ======================================================================================
# 2. Ultralytics 내부에 모듈 등록
# ======================================================================================
target_name = 'CBAM_Universal'

for module_list in [tasks, modules]:
    setattr(module_list, target_name, CBAM_Universal)

# 이미 import된 모듈 캐시에 직접 주입 (안전빵)
if 'ultralytics.nn.modules' in sys.modules:
    setattr(sys.modules['ultralytics.nn.modules'], target_name, CBAM_Universal)

print(f"✅ [준비 완료] 모듈 '{target_name}' 시스템 등록됨.")


✅ [준비 완료] 모듈 'CBAM_Universal' 시스템 등록됨.


In [4]:
# ======================================================================================
# 3. YOLOv8m + CBAM 커스텀 YAML 생성
#    - 채널: 192 / 384 / 576 (오류 로그 기반으로 맞춘 값)
# ======================================================================================

cbam_yaml = {
    'nc': 3,  # 클래스 수 (필요하면 여기만 바꾸면 됨)
    'scales': {'m': [0.67, 0.75, 768]},
    'backbone': [
        # [from, number, module, args]
        [-1, 1, 'Conv', [64, 3, 2]],
        [-1, 1, 'Conv', [128, 3, 2]],
        [-1, 3, 'C2f', [128, True]],

        [-1, 1, 'Conv', [256, 3, 2]],
        [-1, 6, 'C2f', [256, True]],

        # CBAM 1 (채널 192)
        [-1, 1, target_name, [192]],  # idx 5

        [-1, 1, 'Conv', [512, 3, 2]],
        [-1, 6, 'C2f', [512, True]],

        # CBAM 2 (채널 384)
        [-1, 1, target_name, [384]],  # idx 8

        [-1, 1, 'Conv', [1024, 3, 2]],
        [-1, 3, 'C2f', [1024, True]],

        # CBAM 3 (채널 576)
        [-1, 1, target_name, [576]],  # idx 11

        [-1, 1, 'SPPF', [1024, 5]]    # idx 12
    ],
    'head': [
        # P5 → P4
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 8], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [512]],

        # P4 → P3
        [-1, 1, 'nn.Upsample', [None, 2, 'nearest']],
        [[-1, 5], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [256]],

        # P3 → P4
        [-1, 1, 'Conv', [256, 3, 2]],
        [[-1, 15], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [512]],

        # P4 → P5
        [-1, 1, 'Conv', [512, 3, 2]],
        [[-1, 11], 1, 'Concat', [1]],
        [-1, 3, 'C2f', [1024]],

        # Detect
        [[18, 21, 24], 1, 'Detect', ['nc']]
    ]
}

yaml_name = 'yolov8m_cbam_real_final.yaml'
with open(yaml_name, 'w') as f:
    yaml.dump(cbam_yaml, f, sort_keys=False)

print(f"✅ [준비 완료] YAML 파일 생성 완료: {yaml_name}")


✅ [준비 완료] YAML 파일 생성 완료: yolov8m_cbam_real_final.yaml


In [5]:
# ======================================================================================
# 4. Roboflow 데이터셋 다운로드
# ======================================================================================

RF_API_KEY = "HG9M6YJZpcCUgAQaKO9v"  # 너가 쓰던 키 그대로
WORKSPACE = "arakon"
PROJECT = "detection-base-hqaeg"
VERSION = 6
FORMAT = "yolov8"

rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT)
version = project.version(VERSION)
dataset = version.download(FORMAT)

data_path = f"{dataset.location}/data.yaml"
print("✅ Roboflow 데이터셋 위치:", dataset.location)
print("✅ data.yaml 경로:", data_path)

loading Roboflow workspace...
loading Roboflow project...
✅ Roboflow 데이터셋 위치: /workspace/detection-base-6
✅ data.yaml 경로: /workspace/detection-base-6/data.yaml


In [ ]:
# ======================================================================================
# 5. YOLO + CBAM 학습 유틸 함수
# ======================================================================================

def train_stage(
    yaml_path: str,
    data_yaml: str,
    imgsz: int,
    batch: int,
    epochs: int,
    patience: int,
    run_name: str,
    device: int = 0,
    workers: int = 4,
    cache = False,
    lr0: float = 0.001,
    lrf: float = 0.01,
    cos_lr: bool = True,
    optimizer: str = "AdamW",
    mosaic: float = 1.0,
    mixup: float = 0.1,
    close_mosaic: int = 10,
    pretrained_ckpt: str = "yolov8m.pt",
):
    """
    YOLOv8 + CBAM 한 스테이지 학습 함수.
    """
    print(f"\n🚀 학습 시작 | imgsz={imgsz}, batch={batch}, epochs={epochs}, cache={cache}, run={run_name}")

    model = YOLO(yaml_path).load(pretrained_ckpt)

    model.train(
        data=data_yaml,
        epochs=epochs,
        patience=patience,
        imgsz=imgsz,
        batch=batch,
        optimizer=optimizer,
        lr0=lr0,
        lrf=lrf,
        cos_lr=cos_lr,
        mosaic=mosaic,
        mixup=mixup,
        close_mosaic=close_mosaic,
        cache=cache,
        device=device,
        workers=workers,
        name=run_name,
        exist_ok=True,
        deterministic=False
    )

    # 메모리 정리
    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f"✅ 학습 종료 | {run_name}")


In [ ]:
# ======================================================================================
# 6. 멀티 스테이지 학습 실행
#    - 640: 상대적으로 여유 → cache 사용 가능 (원하면 True)
#    - 800 / 1280: 메모리 고려해서 cache=False 권장
# ======================================================================================
'''
# Step 1: 640
train_stage(
    yaml_path=yaml_name,
    data_yaml=data_path,
    imgsz=640,
    batch=16,
    epochs=60,
    patience=50,
    run_name="tank_yolo_cbam_640_final",
    device=0,
    workers=4,
    cache=True,   # 여기는 상황 봐서 True/False 조정
)
'''

# Step 2: 800
train_stage(
    yaml_path=yaml_name,
    data_yaml=data_path,
    imgsz=800,
    batch=8,
    epochs=100,
    patience=30,
    run_name="tank_yolo_cbam_800_final",
    device=0,
    workers=4,
    cache=False,  # 800부터는 메모리 터질 수 있어서 False 추천
)

'''# Step 3: 1280
train_stage(
    yaml_path=yaml_name,
    data_yaml=data_path,
    imgsz=1280,
    batch=4,
    epochs=50,
    patience=30,
    run_name="tank_yolo_cbam_1280_final",
    device=0,
    workers=4,
    cache=False,  # 1280은 무조건 False
)
'''
print("\n🎉 모든 스테이지 학습이 완료되었습니다! 🚀")



🚀 학습 시작 | imgsz=640, batch=16, epochs=60, cache=True, run=tank_yolo_cbam_640_final
✅ CBAM 생성됨 | 적용 채널: 192
✅ CBAM 생성됨 | 적용 채널: 384
✅ CBAM 생성됨 | 적용 채널: 576
Transferred 135/484 items from pretrained weights
Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA A40, 45498MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/detection-base-6/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=trai

adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)


       1/60       7.7G      3.234       3.42      3.523         28        640: 100% ━━━━━━━━━━━━ 87/87 4.9it/s 17.8s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 6.6it/s 0.8s
                   all        132        271      0.672     0.0992    0.00433    0.00132

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)


       2/60       7.7G      2.543      2.753      2.978         39        640: 100% ━━━━━━━━━━━━ 87/87 5.2it/s 16.6s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 5.1it/s 1.0s
                   all        132        271     0.0232      0.242     0.0226    0.00685

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)


       7/50      7.89G      2.026      2.654      2.622         13       1280: 100% ━━━━━━━━━━━━ 346/346 5.3it/s 1:05
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 7.9it/s 2.2s
                   all        132        271      0.337      0.361      0.275      0.118

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)


       8/50      7.96G      1.977      2.602       2.58         13       1280: 100% ━━━━━━━━━━━━ 346/346 5.3it/s 1:05
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 17/17 8.0it/s 2.1s
                   all        132        271      0.363      0.375       0.31      0.136

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


adaptive_max_pool2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
